# Training a ResNet v2 on CIFAR10 dataset

In [2]:
#Identity Mappings in Deep Residual Networks
#https://arxiv.org/pdf/1603.05027.pdf


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.layers.merge import add
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.models import Model
from keras.datasets import cifar10
from keras.utils import plot_model
from keras.utils import to_categorical
import numpy as np
import os

Using TensorFlow backend.


In [4]:
# training parameters
batch_size = 32 
epochs = 200
data_augmentation = True
num_classes = 10

# subtracting pixel mean improves accuracy
subtract_pixel_mean = True

In [11]:
#ResNet56 - v2
n = 6
depth = n*9+2 #56
model_type = 'ResNet%dv2' % (depth)

In [12]:
model_type

'ResNet56v2'

In [13]:
# load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [14]:
# input image dimensions.
input_shape = x_train.shape[1:]

# normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# if subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [15]:
#Learning Rate Schedule
#    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [18]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [19]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])

        num_filters_in = num_filters_out

    # add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model



In [23]:
model = resnet_v2(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
#plot_model(model, to_file="%s.png" % model_type, show_shapes=True)
print(model_type)

Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_175 (Conv2D)             (None, 32, 32, 16)   448         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_166 (BatchN (None, 32, 32, 16)   64          conv2d_175[0][0]                 
__________________________________________________________________________________________________
activation_166 (Activation)     (None, 32, 32, 16)   0           batch_normalization_166[0][0]    
_______________________________________________________________________________________

In [24]:
# prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [25]:
# prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [26]:
# run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # this will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        steps_per_epoch=len(x_train)//batch_size,
                        callbacks=callbacks)

Using real-time data augmentation.
Epoch 1/200
Learning rate:  0.001
1562/1562 [==============================] - 105s 67ms/step - loss: 2.0590 - acc: 0.4968 - val_loss: 1.8523 - val_acc: 0.5322

Epoch 00001: val_acc improved from -inf to 0.53220, saving model to /home/santosh/DL/keras_practice/CIFAR-10/saved_models/cifar10_ResNet56v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1562/1562 [==============================] - 92s 59ms/step - loss: 1.4298 - acc: 0.6362 - val_loss: 1.5046 - val_acc: 0.5832

Epoch 00002: val_acc improved from 0.53220 to 0.58320, saving model to /home/santosh/DL/keras_practice/CIFAR-10/saved_models/cifar10_ResNet56v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
1562/1562 [==============================] - 91s 59ms/step - loss: 1.2178 - acc: 0.6945 - val_loss: 2.0022 - val_acc: 0.5376

Epoch 00003: val_acc did not improve from 0.58320
Epoch 4/200
Learning rate:  0.001
1562/1562 [==============================] - 93s 59ms/step - loss: 1.1075 - acc: 0.7250 -

1562/1562 [==============================] - 90s 58ms/step - loss: 0.5787 - acc: 0.8776 - val_loss: 0.8948 - val_acc: 0.7982

Epoch 00034: val_acc did not improve from 0.84600
Epoch 35/200
Learning rate:  0.001
1562/1562 [==============================] - 90s 58ms/step - loss: 0.5734 - acc: 0.8774 - val_loss: 1.1643 - val_acc: 0.7348

Epoch 00035: val_acc did not improve from 0.84600
Epoch 36/200
Learning rate:  0.001
1562/1562 [==============================] - 90s 58ms/step - loss: 0.5673 - acc: 0.8822 - val_loss: 0.7012 - val_acc: 0.8427

Epoch 00036: val_acc did not improve from 0.84600
Epoch 37/200
Learning rate:  0.001
1562/1562 [==============================] - 90s 58ms/step - loss: 0.5648 - acc: 0.8820 - val_loss: 0.9267 - val_acc: 0.7796

Epoch 00037: val_acc did not improve from 0.84600
Epoch 38/200
Learning rate:  0.001
1562/1562 [==============================] - 90s 58ms/step - loss: 0.5603 - acc: 0.8838 - val_loss: 0.6608 - val_acc: 0.8508

Epoch 00038: val_acc improved 

1562/1562 [==============================] - 91s 59ms/step - loss: 0.4962 - acc: 0.9009 - val_loss: 0.7419 - val_acc: 0.8324

Epoch 00070: val_acc did not improve from 0.87730
Epoch 71/200
Learning rate:  0.001
1562/1562 [==============================] - 91s 58ms/step - loss: 0.4882 - acc: 0.9034 - val_loss: 0.6841 - val_acc: 0.8503

Epoch 00071: val_acc did not improve from 0.87730
Epoch 72/200
Learning rate:  0.001
1562/1562 [==============================] - 91s 59ms/step - loss: 0.4877 - acc: 0.9035 - val_loss: 0.6191 - val_acc: 0.8615

Epoch 00072: val_acc did not improve from 0.87730
Epoch 73/200
Learning rate:  0.001
1562/1562 [==============================] - 92s 59ms/step - loss: 0.4848 - acc: 0.9034 - val_loss: 0.6247 - val_acc: 0.8633

Epoch 00073: val_acc did not improve from 0.87730
Epoch 74/200
Learning rate:  0.001
1562/1562 [==============================] - 92s 59ms/step - loss: 0.4859 - acc: 0.9044 - val_loss: 0.7351 - val_acc: 0.8333

Epoch 00074: val_acc did not i

1562/1562 [==============================] - 94s 60ms/step - loss: 0.2358 - acc: 0.9706 - val_loss: 0.4197 - val_acc: 0.9219

Epoch 00105: val_acc did not improve from 0.92190
Epoch 106/200
Learning rate:  0.0001
1562/1562 [==============================] - 94s 60ms/step - loss: 0.2324 - acc: 0.9710 - val_loss: 0.4185 - val_acc: 0.9220

Epoch 00106: val_acc improved from 0.92190 to 0.92200, saving model to /home/santosh/DL/keras_practice/CIFAR-10/saved_models/cifar10_ResNet56v2_model.106.h5
Epoch 107/200
Learning rate:  0.0001
1562/1562 [==============================] - 93s 60ms/step - loss: 0.2266 - acc: 0.9729 - val_loss: 0.4245 - val_acc: 0.9180

Epoch 00107: val_acc did not improve from 0.92200
Epoch 108/200
Learning rate:  0.0001
1562/1562 [==============================] - 92s 59ms/step - loss: 0.2275 - acc: 0.9722 - val_loss: 0.4139 - val_acc: 0.9214

Epoch 00108: val_acc did not improve from 0.92200
Epoch 109/200
Learning rate:  0.0001
1562/1562 [==============================


Epoch 00140: val_acc did not improve from 0.92420
Epoch 141/200
Learning rate:  1e-05
1562/1562 [==============================] - 91s 58ms/step - loss: 0.1759 - acc: 0.9847 - val_loss: 0.4019 - val_acc: 0.9249

Epoch 00141: val_acc improved from 0.92420 to 0.92490, saving model to /home/santosh/DL/keras_practice/CIFAR-10/saved_models/cifar10_ResNet56v2_model.141.h5
Epoch 142/200
Learning rate:  1e-05
1562/1562 [==============================] - 91s 59ms/step - loss: 0.1760 - acc: 0.9850 - val_loss: 0.4046 - val_acc: 0.9236

Epoch 00142: val_acc did not improve from 0.92490
Epoch 143/200
Learning rate:  1e-05
1562/1562 [==============================] - 91s 59ms/step - loss: 0.1772 - acc: 0.9839 - val_loss: 0.4040 - val_acc: 0.9237

Epoch 00143: val_acc did not improve from 0.92490
Epoch 144/200
Learning rate:  1e-05
1562/1562 [==============================] - 92s 59ms/step - loss: 0.1761 - acc: 0.9848 - val_loss: 0.4053 - val_acc: 0.9235

Epoch 00144: val_acc did not improve from 0.

1562/1562 [==============================] - 91s 58ms/step - loss: 0.1685 - acc: 0.9860 - val_loss: 0.4038 - val_acc: 0.9242

Epoch 00179: val_acc did not improve from 0.92490
Epoch 180/200
Learning rate:  1e-06
1562/1562 [==============================] - 91s 58ms/step - loss: 0.1682 - acc: 0.9866 - val_loss: 0.4041 - val_acc: 0.9244

Epoch 00180: val_acc did not improve from 0.92490
Epoch 181/200
Learning rate:  1e-06
1562/1562 [==============================] - 91s 58ms/step - loss: 0.1681 - acc: 0.9867 - val_loss: 0.4042 - val_acc: 0.9248

Epoch 00181: val_acc did not improve from 0.92490
Epoch 182/200
Learning rate:  5e-07
1562/1562 [==============================] - 91s 58ms/step - loss: 0.1668 - acc: 0.9870 - val_loss: 0.4057 - val_acc: 0.9238

Epoch 00182: val_acc did not improve from 0.92490
Epoch 183/200
Learning rate:  5e-07
1562/1562 [==============================] - 91s 58ms/step - loss: 0.1681 - acc: 0.9864 - val_loss: 0.4049 - val_acc: 0.9245

Epoch 00183: val_acc did n

In [27]:
# score trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 687us/step
Test loss: 0.4056719221115112
Test accuracy: 0.9231
